In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()

In [ ]:
digits = datasets.load_digits()

In [ ]:
 print(digits.data)  

In [ ]:
 digits.target

In [ ]:
digits.images[0]

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

In [ ]:
clf.fit(digits.data[:-1], digits.target[:-1])

In [ ]:
clf.predict(digits.data[-1:])

In [ ]:
import pickle

In [ ]:
s = pickle.dumps(clf)

In [ ]:
clf2 = pickle.loads(s)

In [ ]:
clf2.predict(digits.data[-2:])

### 2.Use Xgboost

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

If you want to use svmlight for less memory consumption, first dumpthe numpy array into svmlight format and then just
pass the filename to DMatrix:

In [ ]:
from sklearn.datasets import dump_svmlight_file
dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

In [ ]:
param = {
'max_depth': 3, # the maximum depth of each tree
'eta': 0.3, # the training step for each iteration
'silent': 1, # logging mode - quiet
'objective': 'multi:softprob', # error evaluation for multiclass training
'num_class': 3} # the number of classes that exist in this datset
num_round = 20 # the number of training iterations

In [ ]:
bst = xgb.train(param, dtrain, num_round)

In [ ]:
preds = bst.predict(dtest)

In [ ]:
import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])

In [ ]:
from sklearn.metrics import precision_score
print (precision_score(y_test, best_preds, average='macro'))

In [ ]:
from sklearn.externals import joblib
joblib.dump(bst, 'bst_model.pkl', compress=True)
# bst = joblib.load('bst_model.pkl') # load it later